In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# 🌦️ Weather forecasting -- _Predictions_

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/4-predictions.ipynb)

This sample is broken into the following notebooks:

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🧭 Overview**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/1-overview.ipynb):
  Go through what we want to achieve, and explore the data we want to use as _inputs and outputs_ for our model.

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🗄️ Create the dataset**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/2-dataset.ipynb):
  Use [Apache Beam](https://beam.apache.org/) to fetch data from [Earth Engine](https://earthengine.google.com/) in parallel, and create a dataset for our model in [Dataflow](https://cloud.google.com/dataflow).

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🧠 Train the model**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/3-training.ipynb):
  Build a simple _Fully Convolutional Network_ in [PyTorch](https://pytorch.org/) and train it in [Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training) with the dataset we created.

* ![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🔮 Model predictions**:
  Get predictions from the model with data it has never seen before.

This sample leverages geospatial satellite and precipitation data from [Google Earth Engine](https://earthengine.google.com/).
Using satellite imagery, you'll build and train a model for rain "nowcasting" i.e. predicting the amount of rainfall for a given geospatial region and time in the immediate future.

* ⏲️ **Time estimate**: ~15 minutes
* 💰 **Cost estimate**: [covered by Cloud Run free tier](https://cloud.google.com/run/pricing)

💚 This is one of many **machine learning how-to samples** inspired from **real climate solutions** aired on the [People and Planet AI 🎥 series](https://www.youtube.com/playlist?list=PLIivdWyY5sqI-llB35Dcb187ZG155Rs_7).

# 🎬 Before you begin

Let's start by cloning the GitHub repository, and installing some dependencies.

In [ ]:
# Now let's get the code from GitHub and navigate to the sample.
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
%cd python-docs-samples/people-and-planet-ai/weather-forecasting

The [`weather-data`](serving/weather-data) local package contains the functions to get data from Earth Engine.
It is used for both creating the training dataset, and for predictions.

The [`weather-model`](serving/weather-model) local package contains the model definition and the training script.
This ensures we use the same model definition for both training and predictions.

We need both of these local modules for predictions.


In [ ]:
# Upgrade `setuptools` to install packages from pyproject.toml files.
!pip install --quiet --upgrade --no-warn-conflicts pip setuptools

# We need `build` and `virtualenv` to build the local packages.
!pip install --quiet build virtualenv

# Install the `weather-data` and `weather-model` local packages.
!pip install serving/weather-data serving/weather-model

## ☁️ My Google Cloud resources

Make sure you have followed these steps to configure your Google Cloud project:

1. Enable the APIs: _Earth Engine, and Cloud Run_

  <button>

  [Click here to enable the APIs](https://console.cloud.google.com/flows/enableapi?apiid=earthengine.googleapis.com,run.googleapis.com)
  </button>

1. Create or use an existing Cloud Storage bucket.

  <button>

  [Click here to create a new Cloud Storage bucket](https://console.cloud.google.com/storage/create-bucket)
  </button>

1. Register your
  [Compute Engine default service account](https://console.cloud.google.com/iam-admin/iam)
  on Earth Engine.

  <button>

  [Click here to register your service account on Earth Engine](https://signup.earthengine.google.com/#!/service_accounts)
  </button>

Once you have everything ready, you can go ahead and fill in your Google Cloud resources in the following code cell.
Make sure you run it!

In [ ]:
from __future__ import annotations

import os
from google.colab import auth

# Please fill in these values.
project = "" #@param {type:"string"}
bucket = "" #@param {type:"string"}
location = "us-central1" #@param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith('gs://'), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Authenticate to Colab.
auth.authenticate_user()

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ['GOOGLE_CLOUD_PROJECT'] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

# 💻 Local predictions

First, we get the input data for the model.
We get the labels as well, just to compare our model's predictions with what the real precipitation actually was.

In [8]:
from datetime import datetime
from weather.data import get_inputs_patch, get_labels_patch

date = datetime(2019, 9, 2, 18)
point = (-78.322, 25.507)  # (longitude, latitude)
patch_size = 128

inputs = get_inputs_patch(date, point, patch_size)
labels = get_labels_patch(date, point, patch_size)

print(f"inputs : {inputs.dtype} {inputs.shape}")
print(f"labels : {labels.dtype} {labels.shape}")

inputs : float32 (128, 128, 52)
labels : float32 (128, 128, 2)


Here's how the input data looks like.

In [9]:
from visualize import show_inputs

# Show the input data for the example.
show_inputs(inputs)

First, let's see how the Vertex AI model we trained performs.

In [ ]:
model_path_gcs = f"gs://{bucket}/weather/model"

!mkdir -p model
!gsutil cp {model_path_gcs}/* model

In [20]:
from weather.model import WeatherModel

model = WeatherModel.from_pretrained("model")

In [21]:
from visualize import show_outputs

predictions = model.predict(inputs.tolist())
show_outputs(predictions)

The results don't look too bad for the small amount of data and time it took to train.
But remember, we trained it on exactly the same date we used for prediction so it's most likely very biased and won't work as well for different dates.

Now, let's see how the pre-trained model in the [`serving/model/`](serving/model/) directory performs.
We trained this model for hundreds of epochs with around 100,000 examples, so you don't have to 🙂.
That is around 800,000 training examples after data augmentation.

In [22]:
from weather.model import WeatherModel

model_path = "serving/model"
model = WeatherModel.from_pretrained(model_path)

In [23]:
from visualize import show_outputs

predictions = model.predict(inputs.tolist())
show_outputs(predictions)

And for reference, this is how the ground truth looks like for this example.

In [ ]:
# Show the real ground truth for reference.
show_outputs(labels)

# ☁️ Cloud Run predictions

[Cloud Run](https://cloud.google.com/run) allows us to deploy
[_serverless_](https://en.wikipedia.org/wiki/Serverless_computing)
web services with a
[REST API](https://en.wikipedia.org/wiki/Representational_state_transfer).
Cloud Run autoscales from zero workers when there are no requests, to enough workers to handle high traffic of requests.
It's a great and efficient option to host a predictions server without having to have servers up and running all the time.

Cloud Run autoscales from zero workers when there are no requests, to enough workers to handle high traffic of requests.
It's a great and efficient option to host a predictions server without having to have servers up and running all the time.

We deploy our service to Cloud Run
[directly from source code](https://cloud.google.com/run/docs/deploying-source-code)
so we don't need to build the container image first.
Behind the scenes, this command uses Google Cloud [Buildpacks](https://buildpacks.io/) and
[Cloud Build](https://cloud.google.com/build)
to automatically build a container image from our source code in the [`serving`](serving) directory.

In [ ]:
service_name = "weather-forecasting"

!gcloud run deploy {service_name} \
  --source="serving/" \
  --region="{location}" \
  --no-allow-unauthenticated

> 💡 Look at your Cloud Run services: https://console.cloud.google.com/run

When the Cloud Run service is deployed, you'll see some information including the Service URL.

We can also get the URL from the Cloud Console, or with [`gcloud run services describe`](https://cloud.google.com/sdk/gcloud/reference/run/services/describe).

In [ ]:
import subprocess

cmd = ["gcloud", "run", "services", "describe", service_name, f"--region={location}", "--format=get(status.url)"]
model_url = subprocess.run(cmd, check=True, stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
print(f"model_url: {model_url}")

Since we only accept authorized calls in our web service, we also need to authenticate each call.

Colab notebooks aren't tied to a Service Account, so we can't get the identity token here.

Run this command in your **local terminal** or in [**Cloud Shell**](https://shell.cloud.google.com/?show=terminal), and then copy-paste your **🔑 Identity token** from the output of the command into the following variable and run that cell.

```sh
# Run in a terminal and copy-paste the outputs.
gcloud auth print-identity-token
```

In [ ]:
identity_token = "" #@param {type:"string"}

> 💡 Your identity token changes over time, so you might need to generate a new one if you're getting authentication errors.

> 💡 To learn more on how to authenticate to Cloud Run, see the
> [Authentication overview](https://cloud.google.com/run/docs/authenticating/overview) page.

Now let's get some predictions from our Cloud Run service.
The service first gets the input data from Earth Engine, gets the predictions and returns them as JSON.

In [ ]:
import numpy as np
import requests

response = requests.get(
    f"{model_url}/predict/-121.526/39.781/2018",
    headers={"Authorization": f"Bearer {identity_token}"},
    params={"patch-size": 512},
)

response.raise_for_status()
results = response.json()

predictions = np.array(results['predictions'], np.uint8)
print(f"predictions : {predictions.dtype.name} {predictions.shape}")

> 💡 To learn about more options for predictions in Cloud, take a look at the [🌍 Land cover classification](../land-cover-classification) sample.

# ⛵ Further exploration

This notebook demonstrated a simple model to start exploring the problem of weather forecasting using deep neural networks. The model has less than 100k parameters and only a few Conv2D layers to keep training time short. Even so, the model is able to distinguish cloud patterns for broad rain vs no rain detection.

There has been a lot of interesting research work on weather nowcasting recently, especially with [U-Net](https://en.wikipedia.org/wiki/U-Net) style model architectures. If you are interested in diving deeper, here are some articles from Google Research:

*  [Google Research blog on nowcasting](https://ai.googleblog.com/2021/11/metnet-2-deep-learning-for-12-hour.html)
*  [MetNet paper](https://arxiv.org/abs/2003.12140)